# 在 TRL 框架下用 LoRA 微调大语言模型

这个 notebook 展示如何用 LoRA 高效微调大语言模型。LoRA 是一种高效的参数微调方法，有如下优点：
- 不更新预训练模型权重
- 仅在注意力层添加少量低秩分解矩阵作为训练参数
- 基本能减少 90% 训练参数
- 能保留模型原有的能力

本文涵盖这些步骤：
1. 配置开发环境、设定 LoRA 相关配置
2. 准备数据集
3. 使用 `trl` 框架下的 `SFTTrainer` 进行 LoRA 微调
4. 测试模型性能、学习加载 adapter


## 1. 配置开发环境

我们首先需要安装 PyTorch 和 Hugging Face 相关的库，这包括 `trl`、`transformers`、`datasets`。其中 `trl` 基于 `transformers` 和 `datasets`，用以微调模型、进行 RLHF、对齐 LLM 等。

In [ ]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login

login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

## 2. 载入数据集

In [13]:
# Load a sample dataset
from datasets import load_dataset

# TODO: 你也可以用自己的数据集
dataset = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")
dataset

DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 119
    })
})

## 3. 在 `trl` 框架下用 `SFTTrainer` 实现大语言模型的 LoRA 微调

在 `trl` 中，[SFTTrainer](https://huggingface.co/docs/trl/sft_trainer) 通过 [PEFT](https://huggingface.co/docs/peft/en/index) 提供了 LoRA Adapter 的集成。这样的设定有以下几个好处：

1. **高效利用内存**：
   - 仅 Adapter 的参数会保存在 GPU 显存中
   - 原模型参数被冻结，所以可以用低精度载入
   - 这使得在消费级显卡上也可以微调大模型
2. **训练层面**：
   - 原生 PEFT/LoRA 集成，用户开发所需代码量少
   - 支持 QLoRA（量化版 LoRA），可以进一步减少内存使用

3. **Adapter 管理**：
   - 可以方便地训练过程中保存 Adapter
   - 可以方便地把 Adapter 融合进原模型

本文将会进行 LoRA 微调，当然你也可以尝试 4-bit 量化来进一步减少内存使用。配置步骤包含以下几步：
1. 定义好 LoRA 的相关参数（主要是 rank、alpha、dropout）
2. 创建一个 SFTTrainer 的实例
3. 训练模型、保存 adapter 的参数

In [ ]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

由于 `SFTTrainer`  原生支持 `peft`，使用 LoRA 训练 LLM 就变得非常简单。我们需要配置的只有 `LoraConfig`。

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>练习：为微调定义好 LoRA 相关参数</h2>
    <p>从 Hugging Face hub 找一个合适的数据然后微调模型 </p> 
    <p><b>难度等级</b></p>
    <p>🐢 使用默认的 LoRA 超参数直接微调</p>
    <p>🐕 改变一些超参数，学习通过 weights & biases 平台查看</p>
    <p>🦁 改变一些超参数，训练后查看这些改变是否影响了推理性能</p>
</div>

In [ ]:
from peft import LoraConfig

# TODO: 配置 LoRA 参数
# r: 低秩分解的秩，越小则需要训练的参数量越少
rank_dimension = 6
# lora_alpha: 将训练好的参数加到原模型上时的缩放倍数，越大则微调参数作用越明显
lora_alpha = 8
# lora_dropout: LoRA 相关层的 dropout，可以用来应对过拟合
lora_dropout = 0.05

peft_config = LoraConfig(
    r=rank_dimension,  # 一般选择 4 到 32
    lora_alpha=lora_alpha,  # 一般是 rank 的 2 倍
    lora_dropout=lora_dropout,  # Dropout probability for LoRA layers
    bias="none",  # Bias type for LoRA. the corresponding biases will be updated during training.
    target_modules="all-linear",  # 模型哪些层会添加 LoRA Adapter
    task_type="CAUSAL_LM",  # Task type for model architecture
)

此外我们还需定义训练的超参数（`TrainingArguments`）。

In [ ]:
# Training configuration
# Hyperparameters based on QLoRA paper recommendations
args = SFTConfig(
    # Output settings
    output_dir=finetune_name,  # Directory to save model checkpoints
    # Training duration
    num_train_epochs=1,  # Number of training epochs
    # Batch size settings
    per_device_train_batch_size=2,  # Batch size per GPU
    gradient_accumulation_steps=2,  # Accumulate gradients for larger effective batch
    # Memory optimization
    gradient_checkpointing=True,  # Trade compute for memory savings
    # Optimizer settings
    optim="adamw_torch_fused",  # Use fused AdamW for efficiency
    learning_rate=2e-4,  # Learning rate (QLoRA paper)
    max_grad_norm=0.3,  # Gradient clipping threshold
    # Learning rate schedule
    warmup_ratio=0.03,  # Portion of steps for warmup
    lr_scheduler_type="constant",  # Keep learning rate constant after warmup
    # Logging and saving
    logging_steps=10,  # Log metrics every N steps
    save_strategy="epoch",  # Save checkpoint every epoch
    # Precision settings
    bf16=True,  # Use bfloat16 precision
    # Integration settings
    push_to_hub=False,  # Don't push to HuggingFace Hub
    report_to=None,  # Disable external logging
)

配置完毕，我们可以创建 `SFTTrainer` 来训练模型了。

In [ ]:
max_seq_length = 1512  # max sequence length for model and packing of the dataset

# Create SFTTrainer with LoRA configuration
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    peft_config=peft_config,  # LoRA configuration
    max_seq_length=max_seq_length,  # Maximum sequence length
    tokenizer=tokenizer,
    packing=True,  # Enable input packing for efficiency
    dataset_kwargs={
        "add_special_tokens": False,  # Special tokens handled by template
        "append_concat_token": False,  # No additional separator needed
    },
)

通过启动 `train()` 函数，我们开始训练。本次训练包含 3 个 epoch。由于我们用了 PEFT，我们可以在训练过程中或结束后，只保存 adapter 的参数，无需保存原模型参数。

In [ ]:
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

# save model
trainer.save_model()

  0%|          | 0/72 [00:00<?, ?it/s]

TrainOutput(global_step=72, training_loss=1.6402628521124523, metrics={'train_runtime': 195.2398, 'train_samples_per_second': 1.485, 'train_steps_per_second': 0.369, 'total_flos': 282267289092096.0, 'train_loss': 1.6402628521124523, 'epoch': 0.993103448275862})

我们模型使用了 Flash Attention 加速训练。在当前数据集（15k 的样本量）训练了 3 轮，在一个 `g5.2xlarge` 机器上用了 4 小时 14 分钟 36 秒。该机器报价 `1.21$/h`，所以我们总花费仅 `5.3$`。


### 将 LoRA Adapter 融入原模型

训练 LoRA 时，我们只训练 adapter 里的参数，而不训练原模型。所以保存的参数也只有 adapter 里的参数（可能也就 2MB 到 10MB）。然而在部署阶段，你可能需要把 Adapter 融合进原模型：
1. **简化的部署流程**：仅载入一个模型参数文件即可，无需额外载入 adapter 参数文件
2. **推理速度提升**：adapter 引入的计算已经融合进了模型中
3. **框架的兼容性**：更能和服务框架适配

In [ ]:
from peft import AutoPeftModelForCausalLM


# Load PEFT model on CPU
model = AutoPeftModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=args.output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# Merge LoRA and base model and save
merged_model = model.merge_and_unload()
merged_model.save_pretrained(
    args.output_dir, safe_serialization=True, max_shard_size="2GB"
)

## 3. 测试模型、进行推理

训练结束后，我们可能需要测试模型。可以从数据集找一些样本，然后看看模型在这些样本上的性能。

In [30]:
# free the memory again
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

# Load Model with PEFT adapter
tokenizer = AutoTokenizer.from_pretrained(finetune_name)
model = AutoPeftModelForCausalLM.from_pretrained(
    finetune_name, device_map="auto", torch_dtype=torch.float16
)
pipe = pipeline(
    "text-generation", model=merged_model, tokenizer=tokenizer, device=device
)

现在我们找些样本来测试：

In [34]:
prompts = [
    "What is the capital of Germany? Explain why thats the case and if it was different in the past?",
    "Write a Python function to calculate the factorial of a number.",
    "A rectangular garden has a length of 25 feet and a width of 15 feet. If you want to build a fence around the entire garden, how many feet of fencing will you need?",
    "What is the difference between a fruit and a vegetable? Give examples of each.",
]


def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        tokenize=False,
        add_generation_prompt=True,
    )
    outputs = pipe(
        prompt,
    )
    return outputs[0]["generated_text"][len(prompt) :].strip()


for prompt in prompts:
    print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(prompt)}")
    print("-" * 50)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>额外练习：载入自己训练的 LoRA Adapter</h2>
    <p>结束本教程后，你可以使用学到的技术，自己训练一个 LoRA，然后载入 LoRA Adapter</p> 
</div>